Importamos las librerias necesarias

In [1]:
import pandas as pd
import numpy as np
import os

Creamos las variables que guardan las rutas de los archivos fuente de datos para lectura y grabación de los mismos

In [2]:
# Inicializar entorno de rutas en google drive para carga de archivos 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Nombre del directorio de datos para rutas de ejecucion en ENTORNO LOCAL
PATH_read_clima = "clima"
PATH_read_datos = "Diarios_2013-2019"
PATH_write = "datoscsv"
pathclima = os.path.join(PATH_read_clima)
pathdatos = os.path.join(PATH_read_datos)
pathgraba = os.path.join(PATH_write)

In [3]:
# cargar dataframes Version Local
'''
maindf = pd.read_csv(pathdatos+"/Diarios_2013-2019.csv",sep=";",decimal=".")
climadf = pd.read_csv(pathclima +'/CLIMA_2013-2019.csv', delimiter=',', decimal=".")
'''
# cargar dataframes Version Colab en Drive
maindf = pd.read_csv("/content/drive/MyDrive/IABD_Proyecto/20220515_IABD_PRJ_Dev_MIKEL/Diarios_2013-2019/Diarios_2013-2019.csv",sep=";",decimal=".")
climadf = pd.read_csv("/content/drive/MyDrive/IABD_Proyecto/20220515_IABD_PRJ_Dev_MIKEL/clima/CLIMA_2013-2019.csv", delimiter=',', decimal=".")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (10,13,14,15,25,43) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


- Aseguramos que el tipo de datos en campo fecha sea `datetime`
- Aseguramos el uso de la puntuación decimal y la conversión de tipos a numérico de todas aquellas observaciones que sean de esa naturaleza

In [4]:
# Convertir la fecha a formato datetime
climadf['Fecha'] = pd.to_datetime(climadf['Fecha'], format="%d/%m/%y")
columclima = climadf.columns.to_list()
borra3 = 'Fecha'
borra = 'HoraRacha'
columclima = list(filter((borra3).__ne__,columclima))
columclima = list(filter((borra).__ne__,columclima))
for colcli in columclima:
    climadf[colcli] = climadf[colcli].replace(',','.', regex=True)
    climadf[colcli] = climadf[colcli].apply(pd.to_numeric)

- Eliminamos las columnas cuyo porcentaje de Nan , nulos o similares esta por encima del 90%
- Aseguramos que el tipo de datos en campos de texto y fecha sea el que corresponde

In [5]:
maindf.drop([ 'BaP','BbFA','BjFA','BkFA','PST', 'UVB','H2S', 'HNM', 'IcdP', 'HCT','HMN', 'FA', 'DahA', 'CH4', 'BaA'] , axis=1, inplace= True)
maindf.drop([ 'NH3','C6H6','C7H8','C8H10','As', 'Cd','Ni', 'Pb'] , axis=1, inplace= True)
# Convertir las cadenas a formato string
maindf['COD_ESTACION'] = maindf['COD_ESTACION'].astype(str)
maindf['NOM_ESTACION'] = maindf['NOM_ESTACION'].astype(str)
# Convertir la fecha a formato datetime
maindf['FECHA'] = pd.to_datetime(maindf['FECHA'], format="%d/%m/%y")

Aseguramos el uso de la puntuación decimal y la conversión de tipos a numérico de todas aquellas observaciones que sean de esa naturaleza

In [6]:
columnas = maindf.columns.to_list()
borra1 = "COD_ESTACION"
borra2 = 'NOM_ESTACION'
borra3 = 'FECHA'
columnas = list(filter((borra1).__ne__,columnas))
columnas = list(filter((borra2).__ne__,columnas))
columnas = list(filter((borra3).__ne__,columnas))

In [7]:
for col in columnas:
    maindf[col] = maindf[col].replace(',','.', regex=True)
    maindf[col] = maindf[col].apply(pd.to_numeric)

### Fusion de Dataframes de clima y calidad del aire ###
- El campo comun es la fecha.
- Los datos de las columnas de clima se repetiran en todas las fechas coincidentes del dataframe resultante.
- Al tratarse de lecturas de datos climatológicos de toda la ciudad ( dataframe `climadf`), todas las estationes tendrán los mismos datos climáticos , en caso de que originalmente esas observaciones estén vacios en el dataframe `maindf`. 

In [8]:
df = pd.merge(left=maindf, right=climadf, how='left', left_on='FECHA', right_on='Fecha')

Eliminamos las filas correspondientes a la estación Valencia Albufera por motivos geograficos ( mediciones fuera del area metropolitana ) y presencia únicamente en algunos años dentro del rango tratado

In [9]:
indexNames = df[ df['COD_ESTACION'] == '46250051' ].index
# Delete these row indexes from dataFrame
df.drop(indexNames , inplace=True)

En el caso de la temperatura los huecos se rellenarán con la media calculada de las temperaturas máxima y mínima que provienen del dataframe de clima (`climadf`)

`Tmedia = df.loc[:,'TempMax':'TempMin'].mean(axis=1)`

`df.loc[df['Temp'].isna(),'Temp']=Tmedia`

- Eliminamos las columnas repetidas y las que porcentaje de Nan , nulos o similares esta por encima del 80%

In [10]:
# actualizar columna 'Temp.' con 'TEMP' cuando el campo 'Temp.' no tenga valor
Tmedia = df.loc[:,'TempMax':'TempMin'].mean(axis=1)
df.loc[df['Temp'].isna(),'Temp']=Tmedia
df.loc[df['lluvias'].isna(),'lluvias']=df['Precip']
df.loc[df['VelocVent'].isna(),'VelocVent']=df['Veloc']
df.loc[df['VelocVentMax'].isna(),'VelocVentMax']=df['RachaMax']
df.drop([ 'Precip','Veloc','RachaMax', 'Fecha','HumRel','Pres','RadSol','Ruido','DirecVent'] , axis=1, inplace= True)

Esta operación permite una reducción drástica de los porcentajes de valores NaN , nulos o similares.

Aumentando así la fiabiliad en los pasos posteriores.

In [11]:
maindf.isnull().sum()/len(maindf)*100

COD_ESTACION     0.000000
NOM_ESTACION     0.000000
FECHA            0.000000
SO2             26.033724
CO              62.630222
NO              23.617227
NO2             23.665557
NOx             23.654817
O3              22.366019
PM10            31.102996
PM2.5           42.648480
PM1             66.083127
DirecVent       67.135646
HumRel          80.431747
lluvias         85.833960
Pres            80.120288
RadSol          80.662657
Ruido           83.460423
Temp            80.104178
VelocVent       66.969176
VelocVentMax    86.827408
dtype: float64

In [12]:
df.isnull().sum()/len(df)*100

COD_ESTACION     0.000000
NOM_ESTACION     0.000000
FECHA            0.000000
SO2             25.524456
CO              62.885469
NO              23.054171
NO2             23.103467
NOx             23.092512
O3              21.821767
PM10            30.245933
PM2.5           41.715506
PM1             65.405050
lluvias          7.377992
Temp             6.791915
VelocVent        4.973435
VelocVentMax     6.939804
HoraRacha        7.854522
TempMax          8.714466
TempMin          8.714466
dtype: float64

In [13]:
df.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
SO2,13597.0,3.426491,1.488254,0.0,3.0,3.0,4.0,24.0
CO,6776.0,0.169569,0.094533,0.0,0.1,0.2,0.2,0.8
NO,14048.0,11.976082,14.526361,0.0,3.0,7.0,15.0,152.0
NO2,14039.0,28.792649,15.953003,0.0,17.0,26.0,38.0,104.0
NOx,14041.0,46.899081,35.978895,0.0,22.0,36.0,60.0,331.0
O3,14273.0,50.077209,19.370573,3.0,37.0,52.0,63.0,193.0
PM10,12735.0,20.072949,11.898731,1.0,12.0,18.0,25.0,161.0
PM2.5,10641.0,12.688385,8.064169,0.0,7.0,11.0,16.0,88.0
PM1,6316.0,8.706935,6.455408,0.0,4.0,7.0,11.0,71.0
lluvias,16910.0,1.067652,5.908871,0.0,0.0,0.0,0.0,118.0


Finalmente guardamos el resultado en un fichero csv para futuras tareas.

In [ ]:
# Guarda datos en CSV:
df.to_csv(pathgraba+'/airWeather_2013-2019.csv', header=True, sep=';', index=False)